# Recommendation Systems

In [1]:
import pandas as pd
import numpy as np

In [2]:
path1  = 'https://github.com/krishnaik06/Recommendation_complete_tutorial/blob/master/KNN%20Movie%20Recommendation/movies.csv?raw=True'
path2 =  'https://github.com/krishnaik06/Recommendation_complete_tutorial/blob/master/KNN%20Movie%20Recommendation/ratings.csv?raw=True'

In [3]:

movies_df = pd.read_csv(path1,usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})
rating_df=pd.read_csv(path2,usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [4]:
#movies_df = pd.read_csv(path1)
#movies_df.head()
#ratings_df = pd.read_csv(path2)
#ratings_df.head()

In [5]:
df = pd.merge( rating_df,movies_df, on = 'movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [6]:
total_rating = df.groupby('title')['rating'].count()
total_ratings = pd.DataFrame(total_rating)
total_ratings.rename(columns ={'rating':'total_rating'},inplace =True)
total_ratings.head()

,total_rating
title,
'71 (2014),1
'Hellboy': The Seeds of Creation (2004),1
'Round Midnight (1986),2
'Salem's Lot (2004),1
'Til There Was You (1997),2


In [7]:
df_final = pd.merge(df,total_ratings, on = 'title')
df_final.head()

,userId,movieId,rating,title,total_rating
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [8]:
minimum_votes = 50 #randomly decided
df_combined = df_final[df_final['total_rating']>50]
df_combined.head()

,userId,movieId,rating,title,total_rating
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [9]:
movie_mat = df_combined.pivot_table(index = 'title', columns = 'userId',values = 'rating').fillna(0)
movie_mat.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [17]:
from scipy.sparse import csr_matrix

movie_mat_matrix = csr_matrix(movie_mat.values)
list_metrics = ['cityblock', 'cosine', 'euclidean', 'l1', 'l2','manhattan','braycurtis', 'canberra', 'chebyshev',
      'correlation', 'dice', 'hamming', 'jaccard', 'kulsinski',
      'mahalanobis', 'minkowski', 'rogerstanimoto', 'russellrao',
      'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean',
      'yule']
from sklearn.neighbors import NearestNeighbors

for i in list_metrics:
    model_knn = NearestNeighbors(metric = i)         #metric was 'cosine' erlier
    model_knn.fit(movie_mat_matrix)
    query_index =416 #np.random.choice(movie_mat.shape[0])
    
    distances, indices = model_knn.kneighbors(movie_mat.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
    for i in range(0, len(distances.flatten())):#Evaluation usin cosine distance
        if i == 0:
            print('Recommendations for {0}:\n'.format(movie_mat.index[query_index]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, movie_mat.index[indices.flatten()[i]], distances.flatten()[i]))
model_knn = NearestNeighbors(metric = 'euclidean')         #metric was 'cosine' erlier
model_knn.fit(movie_mat_matrix)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [24]:
query_index =416 #np.random.choice(movie_mat.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_mat.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

416


In [12]:
for i in range(0, len(distances.flatten())):#Evaluation usin cosine distance
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_mat.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_mat.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Wallace & Gromit: The Wrong Trousers (1993):

1: Chicken Run (2000), with distance of 0.52260422706604:
2: Monty Python and the Holy Grail (1975), with distance of 0.5330187082290649:
3: Monty Python's Life of Brian (1979), with distance of 0.5862387418746948:
4: Toy Story 2 (1999), with distance of 0.5907834768295288:
5: Indiana Jones and the Temple of Doom (1984), with distance of 0.598820149898529:


In [19]:
for i in range(0, len(distances.flatten())): #Evaluation usin euclidean distance
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_mat.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_mat.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Wallace & Gromit: The Wrong Trousers (1993):

1: Crocodile Dundee (1986), with distance of 31.416555404663086:
2: Splash (1984), with distance of 31.42451286315918:
3: Wild Wild West (1999), with distance of 31.432466506958008:
4: Honey, I Shrunk the Kids (1989), with distance of 31.674121856689453:
5: Lost World: Jurassic Park, The (1997), with distance of 32.345787048339844:


## As we can see that using a different metric can significantly change the recommended movies therefore emphasis should be made on which metric to use.

In [26]:
type(distances.flatten())

numpy.ndarray

In [16]:
type(query_index)

int

In [32]:
list_metrics_sparse = ['cityblock', 'cosine', 'euclidean', 'l1', 'l2','manhattan']
list_metrics=['braycurtis', 'canberra', 'chebyshev',
      'correlation', 'dice', 'hamming', 'jaccard', 'kulsinski',
       'minkowski', 'rogerstanimoto', 'russellrao']
from sklearn.neighbors import NearestNeighbors

for i in list_metrics_sparse:
    model_knn = NearestNeighbors(metric = i)         #metric was 'cosine' erlier
    model_knn.fit(movie_mat_matrix)
    query_index =416 #np.random.choice(movie_mat.shape[0])
    
    distances, indices = model_knn.kneighbors(movie_mat.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
    print('****** {} *******'.format(i))
    for i in range(0, len(distances.flatten())):#Evaluation usin cosine distance
        if i == 0:
            print('Recommendations for {0}:\n'.format(movie_mat.index[query_index]))
            
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, movie_mat.index[indices.flatten()[i]], distances.flatten()[i]))

print("**************************************************************************************************")
print("***********************************for non sparsed data********************************************")
print("**************************************************************************************************")
            
for i in list_metrics:
    model_knn = NearestNeighbors(metric = i)         #metric was 'cosine' erlier
    model_knn.fit(movie_mat.values)
    query_index =416 #np.random.choice(movie_mat.shape[0])
    
    distances, indices = model_knn.kneighbors(movie_mat.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
    print('****** {} *******'.format(i))
    for i in range(0, len(distances.flatten())):#Evaluation usin cosine distance
        if i == 0:
            print('Recommendations for {0}:\n'.format(movie_mat.index[query_index]))
            
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, movie_mat.index[indices.flatten()[i]], distances.flatten()[i]))            

****** cityblock *******
Recommendations for Wallace & Gromit: The Wrong Trousers (1993):

1: Splash (1984), with distance of 265.0:
2: Crocodile Dundee (1986), with distance of 267.0:
3: Lady and the Tramp (1955), with distance of 274.0:
4: Wild Wild West (1999), with distance of 275.0:
5: Rocky Horror Picture Show, The (1975), with distance of 280.5:
****** cosine *******
Recommendations for Wallace & Gromit: The Wrong Trousers (1993):

1: Chicken Run (2000), with distance of 0.5226041078567505:
2: Monty Python and the Holy Grail (1975), with distance of 0.5330187082290649:
3: Monty Python's Life of Brian (1979), with distance of 0.5862388014793396:
4: Toy Story 2 (1999), with distance of 0.5907834768295288:
5: Indiana Jones and the Temple of Doom (1984), with distance of 0.598820149898529:
****** euclidean *******
Recommendations for Wallace & Gromit: The Wrong Trousers (1993):

1: Crocodile Dundee (1986), with distance of 31.416555404663086:
2: Splash (1984), with distance of 31.42